## thredds man

the purpose of this notebook is just to get data from thredds and save it so that I don't have to wait for the very fat data every time I test

takes data from `generate_regions_auto` (the pickle files) to slice out smaller regions of data with actual data

larger spans of time are possible now that only smaller regions are being saved

### note

to generate netcdf files for different data, change the `dataset_url` variable and rerun the whole notebook

In [ ]:
%matplotlib inline

In [ ]:
import os
import pickle
import numpy as np
import xarray as xr
from parcels import FieldSet
from datetime import timedelta, datetime

from utils import create_path

In [ ]:
dataset_url_6kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd"
dataset_url_2kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd"
dataset_url_1kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/1km/hourly/RTV/HFRADAR_US_West_Coast_1km_Resolution_Hourly_RTV_best.ncd"
filename_dict = {
    dataset_url_1kmhourly: "west_coast_1km_hourly",
    dataset_url_2kmhourly: "west_coast_2km_hourly",
    dataset_url_6kmhourly: "west_coast_6km_hourly"
}

In [ ]:
### CHANGE THIS TO CHANGE THE DATA RETRIEVED ###
dataset_url = dataset_url_6kmhourly
### ######################################## ###

num_chunks = 50

In [ ]:
netcdf_data = xr.open_dataset(dataset_url, chunks={"time": num_chunks})
netcdf_data

In [ ]:
with open(f"regions_{filename_dict[dataset_url]}.p", "rb") as f:
    regions = pickle.load(f)
regions

## Save slice of time to file

"data size should be less than 55 Mb, but you may be able to get larger requests through without chunking"

In [ ]:
# time_start = 73000
# time_finish = 73083
# sliced_data = netcdf_data.isel(time=slice(time_start, time_finish))
# sliced_data

# get data from a period of time (from latest date going backwards)
timespan = np.timedelta64(7, 'D')
# timespan = np.timedelta64(12,'h')
# time_finish = netcdf_data["time"].max()
# time_start = time_finish - timespan

# this is the timeframe i've been testing
time_start = np.datetime64("2020-06-16T21:00:00.000000000")
time_finish = np.datetime64("2020-06-23T21:00:00.000000000")

sliced_data = netcdf_data.sel(time=slice(time_start, time_finish))
sliced_data

In [ ]:
# megabytes
total_size = sliced_data.nbytes / 1024 / 1024
total_size

In [ ]:
data_regions = np.empty(len(regions), dtype=xr.core.dataset.Dataset)
for i in range(len(regions)):
    new_region = sliced_data.sel(lat=slice(regions[i][0][0], regions[i][0][1]), lon=slice(regions[i][1][0], regions[i][1][1]))
    print(f"region {i} data megabytes: {new_region.nbytes / 1024 / 1024}")
    data_regions[i] = new_region

In [ ]:
save_dir = create_path("current_netcdfs/" + filename_dict[dataset_url])
print(save_dir)

In [ ]:
for i in range(len(data_regions)):
    filename = f"region{i}.nc"
    # save file
    data_regions[i].to_netcdf(save_dir / filename)
    print(f"saved to {save_dir / filename}")
print("done")